In [1]:
from IPython.display import HTML

import json
try:
    import json
except:
    %pip install json
    import json

try:
    import pandas as pd
except:
    %pip install pandas
    import pandas as pd

from operator import itemgetter

import os

from datetime import datetime

try:
    from sortedcontainers import SortedDict

except:
    %pip install sortedcontainers
    from sortedcontainers import SortedDict

try:
    import plotly.express as px
except:
    %pip install plotly.express
    import plotly.express as px



Load data file - events_all.json

In [2]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_ex_by_year = {}

    data = json.load(json_file)
    for event in data.get("events"):

        start = event.get("start")
        year = start.split("-")[0]

        if year in number_ex_by_year:
            number_ex_by_year[year] += 1
        else:
            number_ex_by_year[year] = 1
        
    number_ex_by_year = SortedDict(number_ex_by_year)

    print(number_ex_by_year)

SortedDict({'1929': 2, '1930': 8, '1931': 5, '1932': 7, '1933': 19, '1934': 13, '1935': 8, '1936': 13, '1937': 19, '1938': 13, '1939': 12, '1940': 24, '1941': 43, '1942': 37, '1943': 24, '1944': 21, '1945': 23, '1946': 32, '1947': 21, '1948': 27, '1949': 30, '1950': 28, '1951': 27, '1952': 39, '1953': 50, '1954': 39, '1955': 42, '1956': 44, '1957': 43, '1958': 39, '1959': 64, '1960': 59, '1961': 53, '1962': 42, '1963': 30, '1964': 34, '1965': 37, '1966': 34, '1967': 35, '1968': 39, '1969': 42, '1970': 40, '1971': 41, '1972': 37, '1973': 41, '1974': 48, '1975': 54, '1976': 69, '1977': 81, '1978': 93, '1979': 106, '1980': 81, '1981': 79, '1982': 57, '1983': 75, '1984': 55, '1985': 61, '1986': 70, '1987': 67, '1988': 63, '1989': 50, '1990': 50, '1991': 51, '1992': 41, '1993': 44, '1994': 39, '1995': 46, '1996': 33, '1997': 49, '1998': 61, '1999': 80, '2000': 67, '2001': 51, '2002': 39, '2003': 34, '2004': 52, '2005': 39, '2006': 61, '2007': 73, '2008': 50, '2009': 59, '2010': 48, '2011': 

## Histogram - number of exhibitions by year - all

In [3]:

x_values = number_ex_by_year.keys()
y_values = number_ex_by_year.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Year", "y": "Number of exhibitions"},
title="Number of exhibitions by year"
)
fig.show()

# History - exhibitions per year - non MoMa

In [4]:
filepath = "api"
filename = "events_nonmoma.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_ex_by_year = {}

    data = json.load(json_file)
    for event in data.get("events"):

        start = event.get("start")
        year = start.split("-")[0]

        if year in number_ex_by_year:
            number_ex_by_year[year] += 1
        else:
            number_ex_by_year[year] = 1
        
    number_ex_by_year = SortedDict(number_ex_by_year)

    print(number_ex_by_year)

    x_values = number_ex_by_year.keys()
    y_values = number_ex_by_year.values()

    fig = px.bar(x=x_values, y=y_values, 
    labels={"x": "Year", "y": "Number of exhibitions"},
    title="Number of exhibitions by year - non-MoMa data"
    )
    fig.show()

SortedDict({'1952': 11, '1953': 28, '1954': 23, '1955': 17, '1956': 21, '1957': 26, '1958': 27, '1959': 39, '1960': 37, '1961': 27, '1962': 19, '1963': 10, '1964': 11, '1965': 6, '1990': 1})


# A histogram of number of exhibitions per institution !(MoMA+PS1+whitney) 

In [5]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_by_org = {}

    data = json.load(json_file)
    for event in data.get("events"):
        carried_out_by = event.get("carried_out_by")
        for org in carried_out_by: 
            if "Person" in org:
                continue
           

            # get label for id
            org_file = "../data/group/" + org.split("/").pop() + ".json"
            with open(os.path.join(org_file), 'r') as org_file:
                orgdata = json.load(org_file)

                org_label = orgdata.get("_label")

            if org_label in number_by_org:
                number_by_org[org_label] += 1
            else:
                number_by_org[org_label] = 1
        
    number_by_org = SortedDict(number_by_org)

    
    # sort by number
    number_by_org = dict(sorted(number_by_org.items(), key=lambda item: item[1]))

print(number_by_org)


{'Art Advisory Service': 1, 'Berliner Metallgewerbe Josef Müller': 1, 'Junior Council of The Museum of Modern Art': 2, 'Art Lending Service, The Museum of Modern Art, New York': 5, 'The City Gallery': 8, 'Judson Gallery': 16, 'Reuben Gallery': 21, 'Brata Gallery': 22, 'Green Gallery': 51, 'Hansa Gallery': 84, 'Tanager Gallery': 101, 'MoMA PS1': 1045, 'The Museum of Modern Art': 2439}


In [6]:
x_values = number_by_org.keys()
y_values = number_by_org.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Org", "y": "Number of exhibitions"},
title="Number of exhibitions by org"
)
fig.show()

# remove museum of modern art and moma PS1
 

In [7]:
if "The Museum of Modern Art" in number_by_org:
    del number_by_org["The Museum of Modern Art"]
if "MoMA PS1" in number_by_org:
    del number_by_org["MoMA PS1"]

In [8]:
# sort by number
number_by_org = dict(sorted(number_by_org.items(), key=lambda item: item[1]))

x_values = number_by_org.keys()
y_values = number_by_org.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Org", "y": "Number of exhibitions"},
title="Number of exhibitions by org"
)
fig.show()


# Number of exhibitions per venue

In [9]:
filepath = "api"
filename = "events_all.json"



with open(os.path.join(filepath, filename), 'r') as json_file:
    events_venue = {}
    number_dict = {}

    data = json.load(json_file)
    for event in data.get("events"):
        venue = event.get("venue")

        
        
        if venue in number_dict:
            number_dict[venue] += 1
            events_venue[venue].append(event)
        else:
            number_dict[venue] = 1
            events_venue[venue] = [event]
        
    number_dict = SortedDict(number_dict)

    
    # sort by number
    number_dict = dict(sorted(number_dict.items(), key=lambda item: item[1]))

    

    for venue in events_venue:
        
        venue_filename = venue.replace(" ", "")
        venue_filename = venue_filename.replace(".", "")
        venue_filename = venue_filename.replace("/", "")
        with open('data/summary/venue/exhibitions/' + venue_filename + '.json', 'w') as ex_file:
            ex_file.write(json.dumps({"meta": {"venue": venue, "exhibition_count": number_dict[venue]},"events": events_venue[venue]}))


    venue_index = []

    for venue in number_dict:
        venue_filename = venue.replace(" ", "")
        venue_filename = venue_filename.replace(".", "")
        venue_filename = venue_filename.replace("/", "")
        venue_index.append({"name": venue, "filename": venue_filename,  "number_exhibitions": number_dict[venue]})

    with open('data/summary/venue/exhibitions/index.json', 'w') as ex_file:
            ex_file.write(json.dumps({"venues": venue_index}))

    

In [10]:
x_values = number_dict.keys()
y_values = number_dict.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Venue", "y": "Number of exhibitions"},
title="Number of exhibitions by venue"
)
fig.show()

In [11]:
if "MoMA" in number_dict:
    del number_dict["MoMA"]
if "MoMA PS1" in number_dict:
    del number_dict["MoMA PS1"]

In [12]:
x_values = number_dict.keys()
y_values = number_dict.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Venue", "y": "Number of exhibitions"},
title="Number of exhibitions by venue"
)
fig.show()

# Number of artists per Nationality

In [13]:
filepath = "api"
filename = "persons_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_dict = {}

    data = json.load(json_file)
    for event in data.get("persons"):

        if "nationality" in event:
            nat = event.get("nationality")

            if nat == "":
                continue
            if nat == "American":
                continue
            if nat in number_dict:
                number_dict[nat] += 1
            else:
                number_dict[nat] = 1
        
    number_dict = dict(sorted(number_dict.items(), key=lambda item: item[1]))

    # sort by number
    
    print(number_dict)

{'Greek': 2, 'Egyptian': 3, 'Indian': 3, 'Norwegian': 4, 'Belgian': 4, 'Portuguese': 4, 'Irish': 4, 'Australian': 5, 'Hungarian': 9, 'Chinese': 9, 'Mexican': 11, 'Austrian': 13, None: 13, 'Danish': 14, 'Polish': 14, 'Czech': 14, 'Swiss': 17, 'Swedish': 19, 'Spanish': 20, 'Canadian': 21, 'Dutch': 22, 'Russian': 22, 'Italian': 34, 'Japanese': 39, 'French': 76, 'British': 83, 'German': 99, 'Female': 1610, 'Male': 7145}


In [14]:
x_values = number_dict.keys()
y_values = number_dict.values()

fig = px.bar(x=x_values, y=y_values, 
labels={"x": "Nationality", "y": "Number of people"},
title="Number of people involved in exhibitions by nationality (excluding Americans and empty values)"
)
fig.show()

# Number of exhibitions per year for each organisation

In [15]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    number_ex_by_year = {}

    data = json.load(json_file)
    for event in data.get("events"):

        org = event.get("org")
        start = event.get("start")
        year = start.split("-")[0]

        if org in number_ex_by_year:

            if year in number_ex_by_year[org]:
                number_ex_by_year[org][year] += 1
            else:
                number_ex_by_year[org][year] = 1
        else:
            number_ex_by_year[org] = {}
            number_ex_by_year[org][year] = 1
        
    #number_ex_by_year = SortedDict(number_ex_by_year)

    

    for org in number_ex_by_year:

        numbers = number_ex_by_year[org]
        numbers = SortedDict(numbers)
        print(numbers)
        x_values = numbers.keys()
        y_values = numbers.values()

        fig = px.bar(x=x_values, y=y_values, 
        labels={"x": "Year", "y": "Number of exhibitions"},
        title="Number of exhibitions by year - " + org
        )
        fig.show()
        

SortedDict({'1971': 1, '1973': 8, '1974': 8, '1975': 17, '1976': 21, '1977': 40, '1978': 41, '1979': 58, '1980': 56, '1981': 46, '1982': 39, '1983': 45, '1984': 32, '1985': 28, '1986': 38, '1987': 33, '1988': 27, '1989': 18, '1990': 16, '1991': 15, '1992': 12, '1993': 9, '1994': 6, '1995': 6, '1996': 5, '1997': 12, '1998': 23, '1999': 41, '2000': 25, '2001': 36, '2002': 24, '2003': 22, '2004': 33, '2005': 13, '2006': 28, '2007': 34, '2008': 11, '2009': 13, '2010': 11, '2011': 19, '2012': 19, '2013': 8, '2014': 14, '2015': 11, '2016': 13, '2017': 8, '2018': 2})


SortedDict({'1929': 2, '1930': 8, '1931': 5, '1932': 7, '1933': 19, '1934': 13, '1935': 8, '1936': 13, '1937': 19, '1938': 13, '1939': 12, '1940': 24, '1941': 43, '1942': 37, '1943': 24, '1944': 21, '1945': 23, '1946': 32, '1947': 21, '1948': 27, '1949': 30, '1950': 28, '1951': 27, '1952': 28, '1953': 22, '1954': 16, '1955': 25, '1956': 23, '1957': 17, '1958': 12, '1959': 25, '1960': 22, '1961': 26, '1962': 23, '1963': 20, '1964': 23, '1965': 31, '1966': 34, '1967': 35, '1968': 39, '1969': 42, '1970': 40, '1971': 40, '1972': 37, '1973': 33, '1974': 40, '1975': 37, '1976': 48, '1977': 41, '1978': 52, '1979': 48, '1980': 25, '1981': 33, '1982': 18, '1983': 30, '1984': 23, '1985': 33, '1986': 32, '1987': 34, '1988': 36, '1989': 32, '1990': 33, '1991': 36, '1992': 29, '1993': 35, '1994': 33, '1995': 40, '1996': 28, '1997': 37, '1998': 38, '1999': 39, '2000': 42, '2001': 15, '2002': 15, '2003': 12, '2004': 19, '2005': 26, '2006': 33, '2007': 39, '2008': 39, '2009': 46, '2010': 37, '2011': 4

SortedDict({'1952': 8, '1953': 12, '1954': 9, '1955': 8, '1956': 10, '1957': 11, '1958': 8, '1959': 11, '1960': 9, '1961': 10, '1962': 5})


SortedDict({'1959': 5, '1960': 12, '1961': 3, '1990': 1})


SortedDict({'1952': 3, '1953': 16, '1954': 14, '1955': 9, '1956': 11, '1957': 13, '1958': 11, '1959': 7})


SortedDict({'1960': 3, '1961': 9, '1962': 12, '1963': 10, '1964': 11, '1965': 6})


SortedDict({'1959': 5, '1960': 7, '1961': 3, '1962': 1})


SortedDict({'1957': 2, '1958': 6, '1959': 5, '1960': 6, '1961': 2, '1962': 1})


SortedDict({'1958': 2, '1959': 6})


# keyword frequency in exhibition title - all


In [16]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    numbers = {}

    data = json.load(json_file)
    
    for event in data.get("events"):
        
        title = event.get("_label")
       
        keywords = title.split(" ")
        for keyword in keywords:
            if keyword in numbers:
                numbers[keyword] += 1
            else:
                numbers[keyword] = 1
    
    numbers = dict(sorted(numbers.items(), key=lambda item: item[1]))
    
        
        
        
    print(numbers)
    
    
        

{'Philippe': 1, 'Starck:': 1, 'Marcos': 1, 'Kurtycz': 1, 'Month': 1, 'Sundays': 1, '1940–1970:': 1, 'Antony': 1, 'Johnsons:': 1, 'Swanlights': 1, '61:': 1, 'Biederman,': 1, 'Gummer,': 1, 'Nigel': 1, 'Hall,': 1, 'Kendrick,': 1, 'Avital': 1, 'Oz': 1, 'Sekula:': 1, 'Geography': 1, 'Lesson:': 1, 'Notes': 1, 'Florine': 1, 'Stettheimer': 1, 'Gods,': 1, 'Heroes,': 1, 'Shepherds': 1, 'Civic': 1, 'l:': 1, 'Rudolf': 1, 'Baranik': 1, 'Stevens': 1, '2004': 1, 'Canopy': 1, 'nARCHITECTS': 1, 'Thresholds/O.M.A.': 1, 'MoMA:': 1, 'Mikhailov:': 1, 'Case': 1, 'Utopia/Utopia': 1, 'Navin': 1, 'Rawanchaikul:': 1, '♥': 1, 'Lassnig': 1, 'Haas:': 1, 'Seventies,': 1, 'Dorothy': 1, 'Vogel': 1, 'Display': 1, 'Shigeru': 1, 'Ban:': 1, 'Arc': 1, 'Piper:': 1, 'Synthesis': 1, 'Intuitions': 1, '1965-2016': 1, 'Helena': 1, 'Hernmarck': 1, 'Lamotte': 1, 'Realm': 1, 'Finial': 1, 'Church': 1, 'Sagrada': 1, 'Familia': 1, 'Gesture': 1, 'Pose:': 1, 'Gerald': 1, 'Murphy': 1, 'Huffman,': 1, 'Rosati,': 1, 'Arnold,': 1, 'White,':

# keyword frequency in exhibition title - non MoMa

In [17]:
filepath = "api"
filename = "events_nonmoma.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    numbers = {}

    data = json.load(json_file)
    
    for event in data.get("events"):
        
        title = event.get("_label")
       
        keywords = title.split(" ")
        for keyword in keywords:
            if keyword in numbers:
                numbers[keyword] += 1
            else:
                numbers[keyword] = 1
    
    numbers = dict(sorted(numbers.items(), key=lambda item: item[1]))
    
        
        
        
    print(numbers)
    
    
        

{'Huffman,': 1, 'Rosati,': 1, 'Arnold,': 1, 'White,': 1, 'Levitan': 1, 'Circus': 1, '(Ironworks/Fotodeath)': 1, 'Ida': 1, 'Fischer': 1, 'Varieties:': 1, '(A': 1, 'Shining': 1, 'Bed),': 1, 'Simone': 1, '(See': 1, 'Saw),': 1, '(Chimneyfire': 1, 'Erasers)': 1, 'Fine': 1, 'Dine,': 1, 'Ratliff,': 1, 'Benton,': 1, 'Todd,': 1, 'Gloria': 1, 'Graves': 1, 'Private': 1, 'Myth': 1, 'Blaine,': 1, 'Bourgeois,': 1, 'Grampton,': 1, 'Porter,': 1, 'Flavin,': 1, 'Judd,': 1, 'Marjorie': 1, 'Lascia:': 1, 'David': 1, 'Seccombe': 1, 'Levitt': 1, 'Oils,': 1, 'Thomas;': 1, 'White': 1, 'Fantastic': 1, 'Creatures': 1, 'Satiric': 1, 'Poses': 1, 'Michaela': 1, 'Weisselberg:': 1, 'Constructions,': 1, 'Pastels,': 1, 'Floor': 1, 'Hanging': 1, 'Pieces': 1, 'Sugarman,': 1, 'Bernice': 1, 'D’Vorzon,': 1, 'Bloom,': 1, 'Button,': 1, 'R.': 1, 'Drexler,': 1, 'Kilstrom,': 1, 'McLean,': 1, 'Raleigh,': 1, 'Johnson,': 1, 'Pearlstein;': 1, 'Stein,': 1, 'Whidden': 1, 'Haugard': 1, 'Edelheidt': 1, 'Joan': 1, 'Jacobs': 1, 'Cajori;':

# Create a forced network diagram

diagram with links between artists and organisations (via inclusion in exhibitions)

In [18]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    nodes = []
    links = []

    data = json.load(json_file)
    for event in data.get("events"):

        org = event.get("org")

        nodes_ids  = [x['id'] for x in nodes]
       
        if org == "The ":
            continue
        if org == "MoMA PS1":
            continue
        if org not in nodes_ids:
            nodes.append({"id": org, "group": 1})

          
        influenced_by = event.get("influenced_by")
        for person in influenced_by:
            if person not in nodes_ids:
                nodes.append({"id": person, "group": 2})

            #links_sources  = [x['source'] for x in links]

            links.append({"source": org, "target": person, "value": 1})

    with open('org_artist_links_nonmoma.json', 'w') as file:
        file.write(json.dumps({"nodes": nodes, "links": links}))
        

In [19]:
filepath = "api"
filename = "events_all.json"

with open(os.path.join(filepath, filename), 'r') as json_file:
    
    nodes = []
    links = []

    data = json.load(json_file)
    for event in data.get("events"):

        org = event.get("org")

        nodes_ids  = [x['id'] for x in nodes]
       
        list_orgs = ['Tanager Gallery', "Brata Gallery", "Hansa Gallery", "MoMA PS1"]
        if org not in list_orgs:
            continue
        
        if org not in nodes_ids:
            nodes.append({"id": org, "group": 1})

        influenced_by = event.get("influenced_by")
        for person in influenced_by:
            if person not in nodes_ids:
                nodes.append({"id": person, "group": 2})

            links.append({"source": org, "target": person, "value": 1})


    list_links_persons  = [x['target'] for x in links]
    list_nodes_ids = [x['id'] for x in nodes]

    nonmoma_links = []
    moma_links = []

    for link in links:
        if link.get("source") not in ["MoMA PS1", "The Museum of Modern Art"]:
            nonmoma_links.append(link)
        else:
            moma_links.append(link)

    list_persons_nonmoma = [x['target'] for x in nonmoma_links]
    list_persons_moma = [x['target'] for x in moma_links]

    # get persons in moma list that are not in nonmoma list and then remove links including them 
    # and remove corresponding nodes

    list_persons_uniquetomoma= []
    for person in list_persons_moma:
        if person not in list_persons_nonmoma:
            list_persons_uniquetomoma.append(person)

    
    i = 0
    while i < len(links):
        link = links[i]
        if "target" in link: 
            person = link.get("target")
            if person in list_persons_uniquetomoma:
                
                del links[i]
        i += 1

    i = 0
 

    with open('org_artist_links_10thstreet.json', 'w') as file:
        file.write(json.dumps({"nodes": nodes, "links": links}))


 
        

Timeline visualisation

In [20]:
# Make JSON data file

filepath = "api"
filename = "events_all.json"

events_list = []

with open(os.path.join(filepath, filename), 'r') as json_file:
    data = json.load(json_file)
    for event in data.get("events"):

        start = event.get("start").split("T")[0]

        start_year = start.split("-")[0]
        start_month = start.split("-")[1]
        start_day = start.split("-")[2]

        end = event.get("start").split("T")[0]

        end_year = start.split("-")[0]
        end_month = start.split("-")[1]
        end_day = start.split("-")[2]
         
        

        display_date = str(start) + " - " + str(end)

        headline = event.get("_label")

        event_vis = {
      "media": {
        "url": "",
        "caption": "",
        "credit": "",
        "thumbnail": ""
      },
      "display_date": display_date,
      "start_date": {
        "year": start_year,
        "month": start_month,
        "day": start_day
      },
      "end_date": {
        "year": end_year,
        "month": end_month,
        "day": end_day
      },
      "text": {
        "headline": headline,
        "text": ""
      },
      "type": "title",
      "background": {  "color": "#3f83e8"  }
    }
        events_list.append(event_vis)


  


with open('events_all_timeline.json', 'w') as file:
        file.write(json.dumps({"events": events_list}))

In [21]:
# Make JSON data file

filepath = "data/summary/venue/exhibitions"
filename = "index.json"



with open(os.path.join(filepath, filename), 'r') as json_file:
    data = json.load(json_file)

    for venue in data.get("venues"):
      events_list = []

      filename_venue = venue.get("filename") + ".json"
      with open(os.path.join(filepath, filename_venue), 'r') as venue_file:
        data = json.load(venue_file)
     

        for event in data.get("events"):

          start = event.get("start").split("T")[0]
          start_year = start.split("-")[0]
          start_month = start.split("-")[1]
          start_day = start.split("-")[2]

          end = event.get("start").split("T")[0]
          end_year = start.split("-")[0]
          end_month = start.split("-")[1]
          end_day = start.split("-")[2]
          display_date = str(start) + " - " + str(end)
          headline = event.get("_label")

          event_vis = {
              "media": {   "url": "",   "caption": "",   "credit": "",  "thumbnail": ""   },
              "display_date": display_date,
              "start_date": {   "year": start_year,  "month": start_month, "day": start_day },
              "end_date": { "year": end_year, "month": end_month, "day": end_day},
              "text": {   "headline": headline,"text": ""  },
              "type": "title",
              "background": {  "color": "#3f83e8"  }
            }
          events_list.append(event_vis)

      
      with open('data/summary/venue/timeline/' + filename_venue , 'w') as file:
        file.write(json.dumps({"events": events_list}))

In [26]:
from IPython.display import HTML

import json
try:
    import json
except:
    %pip install json
    import json



from operator import itemgetter

import os

from datetime import datetime


local_html_file = '<head>\
</head>\
<body>\
        <link title="timeline-styles" rel="stylesheet"  href="https://cdn.knightlab.com/libs/timeline3/latest/css/timeline.css"/>\
        <script src="https://cdn.knightlab.com/libs/timeline3/latest/js/timeline.js"></script>\
            <h1>Timeline of exhibition data</h1>\
        <div id="timeline-embed" style="width: 100%; height: 400px"></div>\
        <script type="text/javascript"> window.timeline = new TL.Timeline("timeline-embed", "./data/summary/venue/timeline/HansaGallery.json");</script>\
</body>'

HTML(local_html_file)

In [27]:
try:
    import ipywidgets
except:
    %pip install ipywidgets
    import ipywidgets

from ipywidgets import Layout, FileUpload 
from IPython.display import display, IFrame, HTML, Image
from IPython.core.display import Javascript        
 
import os

try:
    import json
except:
    %pip install json
    import json 



    
selectOptions = []
selectOptions = [('Please select a venue', '')]

filepath = "data/summary/venue/exhibitions"
filename = "index.json"
filepath_timeline = "data/summary/venue/timeline"



with open(os.path.join(filepath, filename), 'r') as json_file:
    data = json.load(json_file)

    for venue in data.get("venues"):
        filename = venue.get("filename")
        name = venue.get("name")
        selectOptions.append( (name , filename))

local_html_file = ""

def dropdown_eventhandler(change):
    
    filename_venue=change.new
    with open(os.path.join(filepath_timeline, filename_venue + '.json'), 'r') as venue_file:
        local_html_file = '<link title="timeline-styles" rel="stylesheet"  href="https://cdn.knightlab.com/libs/timeline3/latest/css/timeline.css"/><script src="https://cdn.knightlab.com/libs/timeline3/latest/js/timeline.js"></script>'

        local_html_file += '<h1>Timeline of exhibition data ' + filename_venue + '</h1>\
        <div id="timeline-embed-' + venue.get("filename") + '" style="background-color:blue;width: 100%; height: 700px">' + filename_venue +'</div>\
        <script type="text/javascript"> window.timeline = new TL.Timeline("timeline-embed-' + venue.get("filename") + '", "./data/summary/venue/timeline/' + filename_venue + '.json");</script>'
        display(HTML(local_html_file))     
    
   

selectObject = ipywidgets.Dropdown(options=selectOptions)
selectObject.observe(dropdown_eventhandler, names='value')

display(selectObject)





Dropdown(options=(('Please select a venue', ''), ('42nd Venice Biennale', '42ndVeniceBiennale'), ('44th Venice…